In [1]:
from pathlib import Path
import sys
import time
import threading

import matplotlib
import matplotlib.pyplot as plt
from matplotlib.lines import Line2D
import numpy as np
import pandas as pd
%matplotlib inline
import autoclasswrapper as wrapper
import os 

afterall = {}
subfeatures = ['Flow_Duration', 'Total_Fwd_Packets']
os.environ['PATH'] += ":/home/chuongdv/Documents/SourceCode/MachineLearning/autoclass-c"
!touch /home/chuongdv/Documents/SourceCode/MachineLearning/exp/result/autoclass.r-params 
!echo "xref_class_report_att_list = 0, 1 \nreport_mode = \"data\" \ncomment_data_headers_p = true " >> /home/chuongdv/Documents/SourceCode/MachineLearning/exp/result/autoclass.r-params 
for feature in subfeatures:
    runtimeDir = "/home/chuongdv/Documents/SourceCode/MachineLearning/exp/result/" + feature
    if not Path(runtimeDir).exists():
        os.mkdir(runtimeDir)
    os.chdir(runtimeDir)
    
            # Create object to prepare dataset.
    clust = wrapper.Input()

    # Load datasets from tsv files.
    pathData = "/home/chuongdv/Documents/SourceCode/MachineLearning/exp/data/features/" + feature + ".tsv"
    clust.add_input_data(pathData, "real location")
    
    # Prepare input data:
    # - create a final dataframe
    # - merge datasets if multiple inputs
    clust.prepare_input_data()

    # Create files needed by AutoClass.
    clust.create_db2_file()
    clust.create_hd2_file()
    clust.create_model_file()
    clust.create_sparams_file(max_duration = 0)
    clust.create_rparams_file()
    !rm autoclass.r-params
    !cp /home/chuongdv/Documents/SourceCode/MachineLearning/exp/result/autoclass.r-params .
    
    !rm -f autoclass-run-* *.results-bin


    # Search autoclass in path.
    wrapper.search_autoclass_in_path()

    # Create object to run AutoClass.
    run = wrapper.Run()

    # Prepare run script.
    run.create_run_file()

    # Run AutoClass.
    run.run()
        
    timer = 0
    step = 2
    while not Path(runtimeDir + "/autoclass-run-success").exists():
        timer += step
        sys.stdout.write("\r")
        sys.stdout.write(f"Time: {timer} sec.")
        sys.stdout.flush()
        time.sleep(step)
        if Path("autoclass-run-failure").exists():
            print("error")
            break
            
    if Path(runtimeDir + "/autoclass-run-success").exists():
        results = wrapper.Output()
        results.extract_results()
        results.aggregate_input_data()
#         results.write_cdt()
#         results.write_cdt(with_proba=True)
#         results.write_class_stats()
#         results.write_dendrogram()
        result = pd.read_csv(runtimeDir + '/autoclass_out.tsv', encoding="utf-8", header=0, sep='\t', index_col=0)
        label =  pd.read_csv('/home/chuongdv/Documents/SourceCode/MachineLearning/exp/data/features/ProtocolName.tsv', encoding='utf-8', header=0, sep='\t', index_col=0)

        label['cluster'] = result['main-class']
        label.to_csv('app.csv', index = True, header=True)


        listH = []
        listApp = []
        listCluster = []
        def fun(x):
            listCluster.append(x['cluster'].values[0])
            listH.append(x['ProtocolName'].value_counts().max()/x.shape[0])
            listApp.append(x['ProtocolName'].value_counts().idxmax())


        label.groupby('cluster').apply(fun)

        H = sum(listH)/len(listH)
        print(listH)
        print(listApp)
        print(listCluster)
        print(H)

        dict = {'Cluster': listCluster, 'App': listApp, 'H': listH}

        saveResult = pd.DataFrame(dict)

        saveResult.to_csv(runtimeDir + '/resultCal.csv', index = True, header=True)
        
        afterall.update({feature:H})
        
round1 = pd.DataFrame(afterall)
round1.to_csv('/home/chuongdv/Documents/SourceCode/MachineLearning/resl.csv', index = True, header=True)

2020-04-26 21:29:14 INFO     Reading data file '/home/chuongdv/Documents/SourceCode/MachineLearning/exp/data/features/Flow_Duration.tsv' as 'real location' with error 0.01
2020-04-26 21:29:14 INFO     Detected encoding: ascii
2020-04-26 21:29:14 INFO     Found 4916 rows and 2 columns
2020-04-26 21:29:14 DEBUG    Checking column names
2020-04-26 21:29:14 DEBUG    Index name 'index'
2020-04-26 21:29:14 DEBUG    Column name 'Flow_Duration'
2020-04-26 21:29:14 INFO     Checking data format
2020-04-26 21:29:14 INFO     Column 'Flow_Duration'
2020-04-26 21:29:14 INFO     count    4.916000e+03
2020-04-26 21:29:14 INFO     mean     2.700050e+07
2020-04-26 21:29:14 INFO     std      4.043878e+07
2020-04-26 21:29:14 INFO     min      1.000000e+00
2020-04-26 21:29:14 INFO     50%      1.005040e+06
2020-04-26 21:29:14 INFO     max      1.200000e+08
2020-04-26 21:29:14 INFO     ---
2020-04-26 21:29:14 INFO     Final dataframe has 4916 lines and 2 columns
2020-04-26 21:29:14 INFO     Searching for m

Time: 1342 sec.

2020-04-26 21:51:38 INFO     Extracting autoclass results
2020-04-26 21:51:38 INFO     Found 4916 cases classified in 68 classes
2020-04-26 21:51:42 INFO     Aggregating input data
2020-04-26 21:51:42 INFO     Writing classes + probabilities .tsv file
2020-04-26 21:51:43 INFO     Reading data file '/home/chuongdv/Documents/SourceCode/MachineLearning/exp/data/features/Total_Fwd_Packets.tsv' as 'real location' with error 0.01
2020-04-26 21:51:43 INFO     Detected encoding: ascii
2020-04-26 21:51:43 INFO     Found 4916 rows and 2 columns
2020-04-26 21:51:43 DEBUG    Checking column names
2020-04-26 21:51:43 DEBUG    Index name 'index'
2020-04-26 21:51:43 DEBUG    Column name 'Total_Fwd_Packets'
2020-04-26 21:51:43 INFO     Checking data format
2020-04-26 21:51:43 INFO     Column 'Total_Fwd_Packets'
2020-04-26 21:51:43 INFO     count      4916.000000
2020-04-26 21:51:43 INFO     mean        175.268511
2020-04-26 21:51:43 INFO     std        3435.497983
2020-04-26 21:51:43 INFO     min     

[0.1751412429378531, 0.08273381294964029, 0.05909090909090909, 0.16363636363636364, 0.15384615384615385, 0.1744186046511628, 0.08695652173913043, 0.07777777777777778, 0.09815950920245399, 0.1875, 0.06504065040650407, 0.08943089430894309, 0.11538461538461539, 0.07407407407407407, 0.22727272727272727, 0.39097744360902253, 0.27049180327868855, 0.10185185185185185, 0.06422018348623854, 0.20192307692307693, 0.08888888888888889, 0.21686746987951808, 0.07042253521126761, 0.18309859154929578, 0.7659574468085106, 0.13253012048192772, 0.07575757575757576, 0.14705882352941177, 0.16666666666666666, 0.1111111111111111, 0.1016949152542373, 0.1111111111111111, 0.15942028985507245, 0.16923076923076924, 0.11764705882352941, 0.4883720930232558, 0.25, 0.08108108108108109, 0.09302325581395349, 0.5853658536585366, 0.18421052631578946, 0.08108108108108109, 0.9032258064516129, 0.14285714285714285, 0.08823529411764706, 0.09375, 0.08695652173913043, 0.08695652173913043, 0.10526315789473684, 0.25, 0.77777777777

2020-04-26 21:51:43 INFO     AutoClass C executable found in /home/chuongdv/Documents/SourceCode/MachineLearning/autoclass-c/autoclass
2020-04-26 21:51:43 INFO     Writing run file
2020-04-26 21:51:43 INFO     AutoClass C executable found in /home/chuongdv/Documents/SourceCode/MachineLearning/autoclass-c/autoclass
2020-04-26 21:51:43 INFO     AutoClass C version: AUTOCLASS C (version 3.3.6unx)
2020-04-26 21:51:43 INFO     Running clustering...


Time: 450 sec.

2020-04-26 21:59:14 INFO     Extracting autoclass results
2020-04-26 21:59:14 INFO     Found 4916 cases classified in 45 classes
2020-04-26 21:59:16 INFO     Aggregating input data
2020-04-26 21:59:16 INFO     Writing classes + probabilities .tsv file


[0.044101433296582136, 0.0801781737193764, 0.15151515151515152, 0.09004739336492891, 0.08994708994708994, 0.10160427807486631, 0.1411192214111922, 0.1392857142857143, 0.06557377049180328, 0.04918032786885246, 0.12037037037037036, 0.47572815533980584, 0.21649484536082475, 0.10344827586206896, 0.13953488372093023, 0.13580246913580246, 0.125, 0.4473684210526316, 0.06779661016949153, 0.14285714285714285, 0.32558139534883723, 0.15428571428571428, 0.09803921568627451, 0.12, 0.08163265306122448, 0.11904761904761904, 0.3333333333333333, 0.15384615384615385, 0.11428571428571428, 0.14285714285714285, 0.21875, 0.1111111111111111, 0.17857142857142858, 0.25925925925925924, 0.125, 0.15, 0.5263157894736842, 0.15, 0.4375, 0.09090909090909091, 0.1, 0.3333333333333333, 0.16666666666666666, 0.2727272727272727, 0.4]
['CLOUDFLARE', 'FACEBOOK', 'TEAMVIEWER', 'CONTENT_FLASH', 'OFFICE_365', 'MQTT', 'NTP', 'NTP', 'MICROSOFT', 'INSTAGRAM', 'TEAMVIEWER', 'DNS', 'WAZE', 'SSL_NO_CERT', 'SSL_NO_CERT', 'SSL_NO_CERT'

ValueError: If using all scalar values, you must pass an index

In [4]:
        result = pd.read_csv(runtimeDir + '/autoclass_out.tsv', encoding="utf-8", header=0, sep='\t', index_col=0)
        label =  pd.read_csv('/home/chuongdv/Documents/SourceCode/MachineLearning/exp/data/features/ProtocolName.tsv', encoding='utf-8', header=0, sep='\t', index_col=0)

        label['cluster'] = result['main-class']
        label.to_csv('app.csv', index = True, header=True)


        listH = []
        listApp = []
        listCluster = []
        def fun(x):
            listCluster.append(x['cluster'].values[0])
            listH.append(x['ProtocolName'].value_counts().max()/x.shape[0])
            listApp.append(x['ProtocolName'].value_counts().idxmax())


        label.groupby('cluster').apply(fun)

        H = sum(listH)/len(listH)
        print(listH)
        print(listApp)
        print(listCluster)
        print(H)

        dict = {'Cluster': listCluster, 'App': listApp, 'H': listH}

        saveResult = pd.DataFrame(dict)

        saveResult.to_csv(runtimeDir + 'resultCal.csv', index = True, header=True)
        
        

[0.11434511434511435, 0.21212121212121213, 0.064343163538874, 0.2364341085271318, 0.06796116504854369, 0.32894736842105265, 0.1572052401746725, 0.19166666666666668, 0.13194444444444445, 0.08396946564885496, 0.22388059701492538, 0.09243697478991597, 0.11811023622047244, 0.272, 0.0673076923076923, 0.0784313725490196, 0.06422018348623854, 0.0967741935483871, 0.14432989690721648, 0.12790697674418605, 0.18309859154929578, 0.21052631578947367, 0.0958904109589041, 0.08823529411764706, 0.15492957746478872, 0.1643835616438356, 0.17543859649122806, 0.1016949152542373, 0.15942028985507245, 0.1038961038961039, 0.11627906976744186, 0.4883720930232558, 0.10526315789473684, 0.25, 0.5714285714285714, 0.09523809523809523, 0.27586206896551724, 0.13333333333333333, 0.2903225806451613, 0.5, 0.5909090909090909, 0.16, 0.3076923076923077, 0.08695652173913043, 0.7777777777777778, 0.631578947368421, 0.125, 1.0, 0.3076923076923077, 0.125, 0.9230769230769231, 0.16666666666666666, 0.2857142857142857, 0.5, 0.4, 0.